# Assignment 3
Training a neural named entity recognition (NER) tagger 

In [ ]:
# ! pip install alive-progress

In [34]:

import torch
import torch.nn as nn
import os 
import numpy as np
from random import shuffle
from sklearn.metrics import classification_report
from tqdm import tqdm
import pandas as pd
from google.colab import drive
from tqdm.notebook import tqdm_notebook
from IPython.display import display
import warnings
warnings.filterwarnings("ignore")

In [2]:

# general folder path
drive_path = '/content/gdrive'
drive_saving_path = '/content/gdrive/My Drive'
drive.mount(drive_path)

Mounted at /content/gdrive


In [3]:
USER = 'DRIVE'  # 'OR' / 'ROY'
if USER == 'OR':
    main_folder =main_folder = r'C:\MSC\NLP2\HW3'
elif USER == 'DRIVE' :
    main_folder = folder_path = os.path.join(drive_saving_path, 'NLP3')


In this assignment you are required to build a full training and testing pipeline for a neural sequentail tagger for named entities, using LSTM.

The dataset that you will be working on is called ReCoNLL 2003, which is a corrected version of the CoNLL 2003 dataset: https://www.clips.uantwerpen.be/conll2003/ner/

[Train data](https://drive.google.com/file/d/1hG66e_OoezzeVKho1w7ysyAx4yp0ShDz/view?usp=sharing)

[Dev data](https://drive.google.com/file/d/1EAF-VygYowU1XknZhvzMi2CID65I127L/view?usp=sharing)

[Test data](https://drive.google.com/file/d/16gug5wWnf06JdcBXQbcICOZGZypgr4Iu/view?usp=sharing)

As you can see, the annotated texts are labeled according to the IOB annotation scheme, for 3 entity types: Person, Organization, Location.

**Task 1:** Write a funtion for reading the data from a single file (of the ones that are provided above). The function recieves a filepath and then it encodes every sentence individually using a pair of lists, one list contains the words and one list contains the tags. Each list pair will be added to a general list (data), which will be returned back from the function.

## set path's

In [4]:
train_path = os.path.join(main_folder, 'connl03_train.txt')
test_path = os.path.join(main_folder, 'connl03_test.txt')
dev_path = os.path.join(main_folder, 'connl03_dev.txt')

In [5]:
def read_data(filepath):
    data = []
    with open(filepath) as file:
        words = []
        labels = []

        for index, line in enumerate(file, start=1):
            if line != '\n':
                word, label = line.split()
                words.append(word)
                labels.append(label)
            else:
                data.append((words, labels))
                words = []
                labels = []
    
    return data

train = read_data(train_path)
dev = read_data(test_path)
test = read_data(dev_path)

The following Vocab class can be served as a dictionary that maps words and tags into Ids. The UNK_TOKEN should be used for words that are not part of the training data.

In [6]:

UNK_TOKEN = 0


class Vocab:
    def __init__(self):
        """
        tag2id/id2tag  - tags to each other from label to integer number
        n_words - count the # of word in sentence
        """
        self.word2id = {"__unk__": UNK_TOKEN}
        self.id2word = {UNK_TOKEN: "__unk__"}
        self.n_words = 1
        
        self.tag2id = {"O":0, "B-PER":1, "I-PER": 2, "B-LOC": 3, "I-LOC": 4, "B-ORG": 5, "I-ORG": 6}
        self.id2tag = {0:"O", 1:"B-PER", 2:"I-PER", 3:"B-LOC", 4:"I-LOC", 5:"B-ORG", 6:"I-ORG"}
    
    
    def index_words(self, words):
        """
        for given token list get token index in sentence
        """
        word_indexes = [self.index_word(w) for w in words]
        return word_indexes


    def index_tags(self, tags):
        """
        for given label list get label index
        """
        tag_indexes = [self.tag2id[t] for t in tags]
        return tag_indexes
    

    def index_word(self, w):
        """
     
        """
        if w not in self.word2id:
            self.word2id[w] = self.n_words
            self.id2word[self.n_words] = w
            self.n_words += 1
        
        return self.word2id[w]
    

**Task 2:** Write a function prepare_data that takes one of the [train, dev, test] and the Vocab instance, for converting each pair of (words,tags) to a pair of indexes. Each pair should be added to data_sequences, which will be returned back from the function.

In [7]:
vocab = Vocab()

def prepare_data(data, vocab):
    data_sequences = []
    # TODO - your code...
    """
    this loop run on the data, for each sequence we generating tesor to
    contain the token of sequence
    """
    for i_words, i_tags in data:
        
        words_indexes_tensor = torch.tensor(vocab.index_words(i_words), dtype=torch.long)
        tags_indexes_tensor = torch.tensor(vocab.index_tags(i_tags), dtype=torch.long)
        # append data and label tensors
        data_sequences.append((words_indexes_tensor, tags_indexes_tensor))

    return data_sequences, vocab

train_sequences, vocab = prepare_data(train, vocab)
dev_sequences, vocab = prepare_data(dev, vocab)
test_sequences, vocab = prepare_data(test, vocab)

**Task 3:** Write NERNet, a PyTorch Module for labeling words with NER tags. 

*input_size:* the size of the vocabulary

*embedding_size:* the size of the embeddings

*hidden_size:* the LSTM hidden size

*output_size:* the number tags we are predicting for

*n_layers:* the number of layers we want to use in LSTM

*directions:* could 1 or 2, indicating unidirectional or bidirectional LSTM, respectively

The input for your forward function should be a single sentence tensor.

*note:* the embeddings in this section are learned embedding. That means that you don't need to use pretrained embedding like the one used in class. You will use them in part 5

In [8]:

class NERNet(nn.Module):
    
    def __init__(self, input_size, embedding_size, hidden_size, output_size, n_layers, directions):
        super(NERNet, self).__init__()
        # TODO: your code...
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size, hidden_size, n_layers, bidirectional=(True if directions==2 else False))
        self.out = nn.Linear(hidden_size*directions, output_size)

        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.directions = directions


    def forward(self, input_sentence):
        # TODO: your code...
        
        # get sentence token numbers to understand output & input size
        dimension = len(input_sentence)
        
        # initial the hidden to None because none sentence inter
        hidden = None

        # 1. foward input sentence into the embeding
        embedded = self.embedding(input_sentence)

        # 2. foward embedding to LSTM
        lstm_output, _ = self.lstm(embedded.view(dimension, 1, -1), hidden) # The view function is meant to reshape the tensor https://stackoverflow.com/a/48650355/7786691

        # 3. foward to get predictions  - linear transformation to the incoming data
        output = self.out(lstm_output.view(dimension, -1)) 

        return output


## help function to train % evaluate 

In [9]:
def get_model_results(model, test_sequences):
    """
    

    Parameters
    ----------
    model : Torch model  - 
        DESCRIPTION: LSTM model.
    test_sequences : list
        DESCRIPTION: input list of coupels [[word_tensor, lebel_tensor] , ...]
    
    the function get model results
    
    Returns
    -------
    all_test_words_pred : list
    all_test_words_true : list
    binary_test_words_pred : list
    binary_test_words_true : list
    """
    # generate test tokens prediction
    all_test_words_pred = []
    all_test_words_true = []

    # generate test binnary prediction
    binary_test_words_pred = []
    binary_test_words_true = []
    for sentence, labels in test_sequences:
        sentence_tensor = torch.LongTensor(sentence).cuda()
        labels_tensor = torch.LongTensor(labels).cuda()

        _, pred_labels = model(sentence_tensor).T.max(0)

        all_test_words_pred += pred_labels.tolist()
        all_test_words_true += labels.tolist()

        binary_test_words_pred += [1 if i >= 1 else i for i in all_test_words_pred]
        binary_test_words_true += [1 if i >= 1 else i for i in all_test_words_true]
    return all_test_words_pred, all_test_words_true, binary_test_words_pred, binary_test_words_true

**Task 4:** write a training loop, which takes a model (instance of NERNet) and number of epochs to train on. The loss is always CrossEntropyLoss and the optimizer is always Adam.

In [30]:

def train_loop(model, n_epochs, train_sequences):
    #
    all_target_names = ["O", "B-PER", "I-PER", "B-LOC", "I-LOC", "B-ORG", "I-ORG"]
    binary_target_names = ["O", "OTHERS"]
    
    # Loss function
    criterion = nn.CrossEntropyLoss()

    # Optimizer (ADAM is a fancy version of SGD)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
  
    # shuffle data before training phase
    shuffle(train_sequences)
    STEP = 400 
    curr_f1_accuracy_result = 0
    best_f1_accuracy_result = 0
    best_df = pd.DataFrame()
    for e in range(1, n_epochs + 1):
        # print('start ephoc #' + str(e),  flush = True)
        # TODO - your code goes here...
        """
        tqdm - add progress bar
        """
        desc = ('Ephoc #' + str(e))
        for sequence_idx in tqdm_notebook(range(train_sequences.__len__()), desc = desc):
            
            
            # get sentence tokens, and labels 
            sentence, labels = train_sequences[sequence_idx]
            
            # check if there is empty sentence
            if labels.__len__() == 0:
                continue
            
            # insert sentence tokens into tensor
            sentence_tensor = torch.LongTensor(sentence).cuda()
            
            # insert sentence labels into tensor
            labels_tensor = torch.LongTensor(labels).cuda()
            
            # Sets the gradients of all optimized to zero.
            model.zero_grad()
            
            # foward sentence to model
            scores = model(sentence_tensor)
            
            # Computes the gradient of current tensor
            criterion(scores, labels_tensor).backward()
            
            # once the gradients are computed use them to optimize model
            optimizer.step()

        
        # print('finshed ephoc #' + str(e) + ', ephoch results:' , flush = True)
        all_train_words_pred, all_train_words_true, \
        binary_train_words_pred, binary_train_words_true = get_model_results(model, train_sequences)
        train_Results_df = pd.DataFrame(classification_report(all_train_words_true, all_train_words_pred, target_names=all_target_names, output_dict = True))
        curr_f1_accuracy_result = train_Results_df.iloc[2]['accuracy']

        
        if curr_f1_accuracy_result > best_f1_accuracy_result:
          improve_string = 'f1-accuracy-score improve from ' + str(best_f1_accuracy_result) + ' to ' + str(curr_f1_accuracy_result) 
          best_f1_accuracy_result = curr_f1_accuracy_result
          best_df = train_Results_df
        else:
          improve_string = 'f1-accuracy-score did not improve from ' + str(best_f1_accuracy_result)  
        print(improve_string, flush = True)
       
    best_string = 'best-f1-accuracy-score is '+ str(best_f1_accuracy_result)  
    print(best_string, flush = True)
    return best_df


**Task 5:** write an evaluation loop on a trained model, using the dev and test datasets. This function print the true positive rate (TPR), also known as Recall and the opposite to false positive rate (FPR), also known as precision, of each label seperately (7 labels in total), and for all the 6 labels (except O) together. The caption argument for the function should be served for printing, so that when you print include it as a prefix.

In [35]:

def evaluate(model, caption, test_sequences, dev_sequences):
    # TODO - your code goes here
    # from Piazza: https://piazza.com/class/klxc3m1tzqz2o8?cid=59

    all_target_names = ["O", "B-PER", "I-PER", "B-LOC", "I-LOC", "B-ORG", "I-ORG"]
    binary_target_names = ["O", "OTHERS"]
    
    # self.tag2id = {"O":0, "B-PER":1, "I-PER": 2, "B-LOC": 3, "I-LOC": 4, "B-ORG": 5, "I-ORG": 6}
    # self.id2tag = {0:"O", 1:"B-PER", 2:"I-PER", 3:"B-LOC", 4:"I-LOC", 5:"B-ORG", 6:"I-ORG"}
    
    print(f"****************    Results for {caption}    ****************")



    # generate dev tokens prediction 
    all_dev_words_pred = []
    all_dev_words_true = []
    
    # generate dev binnary prediction 
    binary_dev_words_pred = []
    binary_dev_words_true = []

    # get test results
    all_test_words_pred, all_test_words_true, \
        binary_test_words_pred, binary_test_words_true = get_model_results(model, test_sequences)

    # get dev results
    all_dev_words_pred, all_dev_words_true, \
        binary_dev_words_pred, binary_dev_words_true = get_model_results(model, dev_sequences)

    print("Test Results:")
    Test_Results_dict = pd.DataFrame(classification_report(all_test_words_true, all_test_words_pred, target_names=all_target_names,  output_dict = True))
    display(Test_Results_dict.T)
    print("Dev Results:")
    Dev_Results_dict = pd.DataFrame(classification_report(all_dev_words_true, all_dev_words_pred, target_names=all_target_names, output_dict = True))
    display(Dev_Results_dict.T)

    print("Binary Test Results:")
    Binary_Test_Results = pd.DataFrame(classification_report(binary_test_words_true, binary_test_words_pred, target_names=binary_target_names, output_dict = True))
    display(Binary_Test_Results.T)

    print("Binary Dev Results:")
    Binary_Dev_Results  = pd.DataFrame(classification_report(binary_dev_words_true, binary_dev_words_pred, target_names=binary_target_names, output_dict = True))
    display(Binary_Dev_Results.T)

    return 

**Task 6:** Train and evaluate a few models, all with embedding_size=300, and with the following hyper parameters (you may use that as captions for the models as well):

Model 1: (hidden_size: 500, n_layers: 1, directions: 1)

Model 2: (hidden_size: 500, n_layers: 2, directions: 1)

Model 3: (hidden_size: 500, n_layers: 3, directions: 1)

Model 4: (hidden_size: 500, n_layers: 1, directions: 2)

Model 5: (hidden_size: 500, n_layers: 2, directions: 2)

Model 6: (hidden_size: 500, n_layers: 3, directions: 2)

Model 4: (hidden_size: 800, n_layers: 1, directions: 2)

Model 5: (hidden_size: 800, n_layers: 2, directions: 2)

Model 6: (hidden_size: 800, n_layers: 3, directions: 2)

In [31]:


# TODO - your code goes here...
EMBEDDING_SIZE = 300
EPOCHS = 10
HIDDEN_SIZE  = 500 
INPUT_SIZE = len(vocab.word2id) # 8955
OUTPUT_SIZE = len(vocab.tag2id) # 7

n_layers_array = np.arange(1,4)
directions_array = np.arange(1,3)

# n_layers_array = np.arange(1,2)
# directions_array = np.arange(1,2)
model_list  = []
train_res_list = [] 
for i_n_layers in n_layers_array:
    for i_directions in directions_array:
        print('----------------------------------------------------------')
        print('Train model using:\n' + \
              '  1)hidden_size = ' + str(HIDDEN_SIZE)+'\n'+ \
              '  2)n_layers = ' + str(i_n_layers) + '\n'+ \
              '  3)directions = ' + str(i_directions) , flush = True)
        
        model = NERNet(INPUT_SIZE, EMBEDDING_SIZE, HIDDEN_SIZE, OUTPUT_SIZE, int(i_n_layers), int(i_directions)).cuda()
        train_res = train_loop(model, EPOCHS, train_sequences)
        model_list.append(model)
        train_res_list.append(train_res)

DIRECTION = 2
HIDDEN_SIZE = 800
for i_n_layers in n_layers_array:
        print('----------------------------------------------------------')

        print('Train model using:\n'+ \
              '  1)hidden_size = ' + str(HIDDEN_SIZE)+'\n'+ \
              '  2)n_layers = ' + str(i_n_layers) + '\n'+ \
              '  3)directions = ' + str(i_directions) , flush = True )
        model = NERNet(INPUT_SIZE, EMBEDDING_SIZE, HIDDEN_SIZE, OUTPUT_SIZE, i_n_layers, DIRECTION).cuda()
        train_res = train_loop(model, EPOCHS, train_sequences)
        model_list.append(model)

        

----------------------------------------------------------
Train model using:
  1)hidden_size = 500
  2)n_layers = 1
  3)directions = 1


Ephoc #1:   0%|          | 0/1750 [00:00<?, ?it/s]

f1-accuracy-score improve from 0 to 0.8406118143459915


Ephoc #2:   0%|          | 0/1750 [00:00<?, ?it/s]

f1-accuracy-score improve from 0.8406118143459915 to 0.8886075949367088


...

f1-accuracy-score did not improve from 0.995534458509142


Ephoc #9:   0%|          | 0/1750 [00:00<?, ?it/s]

f1-accuracy-score did not improve from 0.995534458509142


Ephoc #10:   0%|          | 0/1750 [00:00<?, ?it/s]

f1-accuracy-score improve from 0.995534458509142 to 0.9971518987341772


## evaluate 


In [36]:
for i, model in enumerate(model_list):
    model_name = "model_"+str(i)
    evaluate(model, model_name, test_sequences, dev_sequences)

****************    Results for model_0    ****************
Test Results:


,precision,recall,f1-score,support
O,0.930814,0.938630,0.934706,3096.000000
B-PER,0.602679,0.675000,0.636792,200.000000
I-PER,0.735714,0.656051,0.693603,157.000000
B-LOC,0.791411,0.704918,0.745665,183.000000
I-LOC,1.000000,0.521739,0.685714,23.000000
B-ORG,0.520833,0.595238,0.555556,168.000000
I-ORG,0.600000,0.465517,0.524272,116.000000
accuracy,0.872179,0.872179,0.872179,0.872179
macro avg,0.740207,0.651013,0.682330,3943.000000
weighted avg,0.873135,0.872179,0.871539,3943.000000


Dev Results:


,precision,recall,f1-score,support
O,0.933424,0.939394,0.936400,6567.000000
B-PER,0.664319,0.652074,0.658140,434.000000
I-PER,0.744526,0.689189,0.715789,296.000000
B-LOC,0.794118,0.708455,0.748844,343.000000
I-LOC,0.931034,0.509434,0.658537,53.000000
B-ORG,0.521739,0.617143,0.565445,350.000000
I-ORG,0.448649,0.415000,0.431169,200.000000
accuracy,0.876501,0.876501,0.876501,0.876501
macro avg,0.719687,0.647241,0.673475,8243.000000
weighted avg,0.877418,0.876501,0.876227,8243.000000


Binary Test Results:


...

,precision,recall,f1-score,support
O,0.946574,0.973961,0.960072,6567.000000
B-PER,0.800512,0.721198,0.758788,434.000000
I-PER,0.817857,0.773649,0.795139,296.000000
B-LOC,0.833333,0.758017,0.793893,343.000000
I-LOC,0.681818,0.566038,0.618557,53.000000
B-ORG,0.702454,0.654286,0.677515,350.000000
I-ORG,0.714286,0.475000,0.570571,200.000000
accuracy,0.916171,0.916171,0.916171,0.916171
macro avg,0.785262,0.703164,0.739219,8243.000000
weighted avg,0.911846,0.916171,0.912993,8243.000000


Binary Test Results:


,precision,recall,f1-score,support
O,0.934804,0.973443,0.953732,380877.000000
OTHERS,0.896685,0.772471,0.829956,113647.000000
accuracy,0.927257,0.927257,0.927257,0.927257
macro avg,0.915744,0.872957,0.891844,494524.000000
weighted avg,0.926044,0.927257,0.925287,494524.000000


Binary Dev Results:


,precision,recall,f1-score,support
O,0.945277,0.974850,0.959836,1.616269e+06
OTHERS,0.891143,0.784862,0.834633,4.239800e+05
accuracy,0.935369,0.935369,0.935369,9.353692e-01
macro avg,0.918210,0.879856,0.897234,2.040249e+06
weighted avg,0.934028,0.935369,0.933818,2.040249e+06


**Task 6:** Download the GloVe embeddings from https://nlp.stanford.edu/projects/glove/ (use the 300-dim vectors from glove.6B.zip). Then intialize the nn.Embedding module in your NERNet with these embeddings, so that you can start your training with pre-trained vectors. Repeat Task 6 and print the results for each model.

Note: make sure that vectors are aligned with the IDs in your Vocab, in other words, make sure that for example the word with ID 0 is the first vector in the GloVe matrix of vectors that you initialize nn.Embedding with. For a dicussion on how to do that, check it this link:
https://discuss.pytorch.org/t/can-we-use-pre-trained-word-embeddings-for-weight-initialization-in-nn-embedding/1222

## move to drive folder

In [60]:
! cd "/content/gdrive/My Drive/NLP3" 
GLOVE_PATH  = os.path.join(main_folder, 'glove.6B.300d.txt')
GLOVE_PATH = 'glove.6B.300d.txt'

In [59]:
!ls

gdrive		   glove.6B.200d.txt  glove.6B.50d.txt	glove.6B.zip.1
glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip	sample_data


## get glove data set

In [57]:
# TODO - your code goes here...
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zipy

--2022-05-24 14:02:50--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-05-24 14:02:50--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-05-24 14:02:50--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove

# get embding weights

In [52]:
def get_glove_pre_trained_embeddings_weights(input_size, embedding_size, word2id = vocab.word2id):
    # generate zeros initilized embeding mask 
    weights = np.zeros((input_size, embedding_size))

    # parse embeding
    with open(GLOVE_PATH) as glove:
        for line in glove.readlines():
            split = line.split()
            word = split[0]
            word_id = word2id.get(word)

        if word_id:
            weights[word_id] = split[1:]
    tensor = torch.from_numpy(weights).float()
    return tensor

## define Glove net

In [53]:
class GloveNERNet(nn.Module):
    
    def __init__(self, input_size, embedding_size, hidden_size, output_size, n_layers, directions):
        super(GloveNERNet, self).__init__()
        
        # TODO: your code...

        # add first layer - the embding layer
        self.embedding = nn.Embedding(input_size, embedding_size)

        # get trained embding weights from the data we loaded
        pre_trained_weights = get_glove_pre_trained_embeddings_weights(input_size, embedding_size)
        
        # load embding weights 
        self.embedding.weight = nn.Parameter(pre_trained_weights)

        # add LSTM layer
        self.lstm = nn.LSTM(embedding_size, hidden_size, n_layers, bidirectional=(True if directions==2 else False))
        
        # Add FC layer
        self.out = nn.Linear(hidden_size*directions, output_size)
    
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.directions = directions

    def forward(self, input_sentence):
        # TODO: your code...
        
        # get sentence token numbers to understand output & input size
        dimension = len(input_sentence)
        
        # initial the hidden to None because none sentence inter
        hidden = None

        # 1. foward input sentence into the embeding
        embedded = self.embedding(input_sentence)

        # 2. foward embedding to LSTM
        lstm_output, _ = self.lstm(embedded.view(dimension, 1, -1), hidden) # The view function is meant to reshape the tensor https://stackoverflow.com/a/48650355/7786691

        # 3. foward to get predictions  - linear transformation to the incoming data
        output = self.out(lstm_output.view(dimension, -1)) 

        return output

## train model

In [63]:


# TODO - your code goes here...
EMBEDDING_SIZE = 300
EPOCHS = 10
HIDDEN_SIZE  = 500 
INPUT_SIZE = len(vocab.word2id) # 8955
OUTPUT_SIZE = len(vocab.tag2id) # 7

n_layers_array = np.arange(1,4)
directions_array = np.arange(1,3)

# n_layers_array = np.arange(1,2)
# directions_array = np.arange(1,2)
model_list  = []
train_res_list = [] 
for i_n_layers in n_layers_array:
    for i_directions in directions_array:
        print('----------------------------------------------------------')
        print('Train model using:\n' + \
              '  1)hidden_size = ' + str(HIDDEN_SIZE)+'\n'+ \
              '  2)n_layers = ' + str(i_n_layers) + '\n'+ \
              '  3)directions = ' + str(i_directions) , flush = True)
        model = GloveNERNet(INPUT_SIZE, EMBEDDING_SIZE, HIDDEN_SIZE, OUTPUT_SIZE, 1, 1).cuda()
        train_res = train_loop(model, EPOCHS, train_sequences)   
        model_list.append(model)
        train_res_list.append(train_res)

DIRECTION = 2
HIDDEN_SIZE = 800
for i_n_layers in n_layers_array:
        print('----------------------------------------------------------')

        print('Train model using:\n'+ \
              '  1)hidden_size = ' + str(HIDDEN_SIZE)+'\n'+ \
              '  2)n_layers = ' + str(i_n_layers) + '\n'+ \
              '  3)directions = ' + str(i_directions) , flush = True )
        model = GloveNERNet(INPUT_SIZE, EMBEDDING_SIZE, HIDDEN_SIZE, OUTPUT_SIZE, 1, 1).cuda()
        train_res = train_loop(model, EPOCHS, train_sequences)   

       
        model_list.append(model)

        

----------------------------------------------------------
Train model using:
  1)hidden_size = 500
  2)n_layers = 1
  3)directions = 1


Ephoc #1:   0%|          | 0/1750 [00:00<?, ?it/s]

f1-accuracy-score improve from 0 to 0.8534458509142053


Ephoc #2:   0%|          | 0/1750 [00:00<?, ?it/s]

f1-accuracy-score improve from 0.8534458509142053 to 0.9197960618846694


...

f1-accuracy-score improve from 0.9854078762306611 to 0.989521800281294


Ephoc #9:   0%|          | 0/1750 [00:00<?, ?it/s]

f1-accuracy-score improve from 0.989521800281294 to 0.9911040787623067


Ephoc #10:   0%|          | 0/1750 [00:00<?, ?it/s]

f1-accuracy-score improve from 0.9911040787623067 to 0.9924050632911392


## evaluate 


In [64]:
for i, model in enumerate(model_list):
    model_name = "model_"+str(i)
    evaluate(model, model_name, test_sequences, dev_sequences)

****************    Results for model_0    ****************
Test Results:


,precision,recall,f1-score,support
O,0.982728,0.900517,0.939828,3096.000000
B-PER,0.906040,0.675000,0.773639,200.000000
I-PER,0.923077,0.611465,0.735632,157.000000
B-LOC,0.939597,0.765027,0.843373,183.000000
I-LOC,0.833333,0.434783,0.571429,23.000000
B-ORG,0.366234,0.839286,0.509946,168.000000
I-ORG,0.302932,0.801724,0.439716,116.000000
accuracy,0.863048,0.863048,0.863048,0.863048
macro avg,0.750563,0.718257,0.687652,3943.000000
weighted avg,0.927324,0.863048,0.883613,3943.000000


Dev Results:


,precision,recall,f1-score,support
O,0.988673,0.890513,0.937029,6567.000000
B-PER,0.892857,0.691244,0.779221,434.000000
I-PER,0.886878,0.662162,0.758221,296.000000
B-LOC,0.939271,0.676385,0.786441,343.000000
I-LOC,0.812500,0.490566,0.611765,53.000000
B-ORG,0.356882,0.837143,0.500427,350.000000
I-ORG,0.256334,0.860000,0.394948,200.000000
accuracy,0.857333,0.857333,0.857333,0.857333
macro avg,0.733342,0.729716,0.681150,8243.000000
weighted avg,0.932190,0.857333,0.882251,8243.000000


Binary Test Results:


...

,precision,recall,f1-score,support
O,0.984941,0.886402,0.933077,6567.000000
B-PER,0.908497,0.640553,0.751351,434.000000
I-PER,0.888889,0.621622,0.731610,296.000000
B-LOC,0.812883,0.772595,0.792227,343.000000
I-LOC,0.560606,0.698113,0.621849,53.000000
B-ORG,0.340144,0.808571,0.478849,350.000000
I-ORG,0.266779,0.795000,0.399497,200.000000
accuracy,0.852481,0.852481,0.852481,0.852481
macro avg,0.680391,0.746122,0.672637,8243.000000
weighted avg,0.922776,0.852481,0.876179,8243.000000


Binary Test Results:


,precision,recall,f1-score,support
O,0.979905,0.901094,0.938849,380877.000000
OTHERS,0.738904,0.938071,0.826660,113647.000000
accuracy,0.909592,0.909592,0.909592,0.909592
macro avg,0.859404,0.919583,0.882754,494524.000000
weighted avg,0.924521,0.909592,0.913067,494524.000000


Binary Dev Results:


,precision,recall,f1-score,support
O,0.983756,0.884647,0.931573,1.616269e+06
OTHERS,0.682280,0.944313,0.792191,4.239800e+05
accuracy,0.897046,0.897046,0.897046,8.970459e-01
macro avg,0.833018,0.914480,0.861882,2.040249e+06
weighted avg,0.921107,0.897046,0.902608,2.040249e+06


**Good luck!**